In [ ]:
# From b_step_up
# get imports
# first e_ notebook for classical ML
# this is for Random Forest predictions on the original 5k mad data
# template for RF (or other classical ML such as DT) on decoded samples

# ops for step-up experiment: read from F1

In [ ]:
 # Original baseline F1 scores on 5k feats for plotting - v5, redoing on M1 machine
plt_v = plt_v        # 15 is step up

re_samp = re_samp
trn_tst_splts = trn_tst_splts

blnk_frm = pd.DataFrame()

for path in paths:
    
    cohort = path.split('/')[1].split('.')[0]
    print(cohort)
    # break
    df = pd.read_csv(path, sep="\t", index_col=0)
    features = df.iloc[:, 1:].mad().sort_values(ascending=False)[:5000].index
    normVals = df[features].max()
    vals = df[features]
    X_5k = (vals / normVals)
    df = pd.concat( [df.Labels, X_5k] , axis = 1) # Overwrite df with 5k mad feats
    # break
    F1_dct = {}
    for j in list(range(0, re_samp)): # 100% sampling rate
        X = df.iloc[:, 1:]
        y = df.iloc[:, 0]
        output = []
        for i in list(range(0, trn_tst_splts)): # number of train test splits
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            clf = RandomForestClassifier(max_depth=2, # reset the classifier each time through loop
                )
            clf.fit(X_train, y_train)
            F1_scr = f1_score(y_test, clf.predict(X_test), average='weighted')
            output.append(F1_scr) # extracting stats and overwriting, str(clf)
                                  
        F1_dct['mean'+str(j)] = mean(output)
        F1_dct['std'+str(j)] = stdev(output)

    chrt_frm = pd.DataFrame(F1_dct, index = [cohort + '_' + str(clf)])
    blnk_frm = pd.concat( [blnk_frm, chrt_frm] )
    # print('path loop done')

print('all done') # no auto-file writing, interactive mode

In [ ]:
blnk_frm.to_csv('raw_baseline_F1s/5k_r13.tsv', sep = '\t')